In [1]:
from transformers import pipeline

model_id = "cjvt/GaMS-2B-Instruct"

pline = pipeline(
    "text-generation",
    model=model_id,
    device_map="auto" 
)


d:\5_programi\590_programming\Miniconda3\envs\nlp-project\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Loading checkpoint shards: 100%|██████████| 2/2 [00:02<00:00,  1.03s/it]
Device set to use cpu


In [ ]:

# Example of response generation
message = [{"role": "user", "content": "Kateri je najpomembnejši dogodek v slovenski zgodovini?"}]
response = pline(message, max_new_tokens=512)
print("Model's response:", response[0]["generated_text"][-1]["content"])


Model's response: Najpomembnejši dogodek v slovenski zgodovini je bil razpad Avstro-Ogrske in priključitev Prekmurja k Sloveniji.



In [9]:

# Example of conversation chain
new_message = response[0]["generated_text"]
new_message.append({"role": "user", "content": "Lahko bolj podrobno opišeš ta dogodek?"})
new_message = new_message[:3]


In [10]:


response = pline(new_message, max_new_tokens=300)
print("Model's response:", response[0]["generated_text"][-1]["content"])


Model's response: Razpad Avstro-Ogrske je bil posledica prve svetovne vojne in razpada Avstro-Ogrskega imperija. Po vojni je bila ustanovljena Kraljevina SHS, ki je vključevala večino slovenskega etničnega ozemlja, vključno s Prekmurjem. Prekmurje je bilo dolgo časa pod madžarsko oblastjo, zato je bilo to priključitev še posebej pomembno za ohranjanje slovenskega jezika in kulture.



In [11]:
prompt = """Classify the text into neutral, negative or positive.
Here are some annotated examples:
Example 1:
Text: 'This restaurant is the best I've ever been to. The food was delicious and the staff were very friendly.'
Sentiment: positive

Example 2:
Text: 'I was disappointed with my purchase. The product broke within a week.'
Sentiment: negative

Example 3:
Text: 'The movie was okay, not great but not bad either.'
Sentiment: neutral

Text: 'I absolutely love the new Spider-Man movie. It's incredibly well done!'
Sentiment:"""
res = pline(prompt)
print(res[0]["generated_text"])


Classify the text into neutral, negative or positive.
Here are some annotated examples:
Example 1:
Text: 'This restaurant is the best I've ever been to. The food was delicious and the staff were very friendly.'
Sentiment: positive

Example 2:
Text: 'I was disappointed with my purchase. The product broke within a week.'
Sentiment: negative

Example 3:
Text: 'The movie was okay, not great but not bad either.'
Sentiment: neutral

Text: 'I absolutely love the new Spider-Man movie. It's incredibly well done!'
Sentiment: positive

Text: 'The weather forecast for the next few days looks pretty good.'
Sentiment:


In [ ]:
import pandas as pd

DATA_PATH = "../data/reports_promet_merged.csv"

data = pd.read_csv(DATA_PATH)


In [17]:
example = data.iloc[0]
example

report_id                                                                 0
report_datetime                                         2022-01-01 06:00:00
report_type                                            Prometne informacije
report_path               ../data/RTVSlo/Podatki - rtvslo.si\Promet 2022...
report_content            Podatki o prometu.\nPonekod po državi megla zm...
info_id                                                             1283526
info_datetime                                           2022-01-01 05:56:41
info_aggregated_urgent                                                  NaN
info_aggregated_basic     Vreme\nPonekod po Sloveniji megla v pasovih zm...
info_urgent                                                             NaN
info_accidents                                                          NaN
info_jams                                                               NaN
info_weather              Ponekod po Sloveniji megla v pasovih zmanjšuje...
info_obstacl

In [ ]:
example["report_content"]

'Podatki o prometu.\nPonekod po državi megla zmanjšuje vidljivost.\nZaradi del je na vzhodni mariborski obvoznici v obe smeri zaprt prehitevalni pas med razcepom Dragučova in priključkom Pesnica.\n'

In [ ]:
example["info_aggregated_basic"]

'Vreme\nPonekod po Sloveniji megla v pasovih zmanjšuje vidljivost. Prilagodite hitrost!\nOmejitve za tovorna vozila\nPo Sloveniji velja med prazniki omejitev za tovorna vozila z največjo dovoljeno maso nad 7,5 ton:\n- danes, 1. 1., od 8. do 22. ure;\n- v nedeljo, 2. 1., od 8. do 22. ure.\nOd 30. decembra je v veljavi sprememba omejitve za tovorna vozila nad 7,5 ton. Več.\nDela\nNa primorski avtocesti je ponovno odprt priključek Črni Kal v obe smeri.'

In [32]:
prompt = """Napiši poročilo iz podanih informacij. V primeru nujnih informacij jih obvezno vključi v poročilo, kot v primerih. 
Podanih imaš nekaj primerov.
"""

examples = data.iloc[3:7]
test_urgent = data.iloc[10]["info_aggregated_urgent"]
test_basic = data.iloc[10]["info_aggregated_basic"]
test_content = data.iloc[10]["report_content"]

for inx, curr in examples.iterrows():
    urgent = curr["info_aggregated_urgent"]
    basic = curr["info_aggregated_basic"]
    content = curr["report_content"]

    example_prompt = f"""
    PRIMER {inx+1}: 
        
        INFORMACIJE:
            Nujne informacije: {urgent}

            Informacije: {basic}

        POROČILO:
            {content}
    """ 

    prompt += example_prompt


add_example = f"""
        INFORMACIJE:
            Nujne informacije: {test_urgent}

            Informacije: {test_basic}

        POROČILO:
    """ 

prompt += add_example

print(prompt)


Napiši poročilo iz podanih informacij. V primeru nujnih informacij jih obvezno vključi v poročilo, kot v primerih. 
Podanih imaš nekaj primerov.

    PRIMER 4: 

        INFORMACIJE:
            Nujne informacije: nan

            Informacije: Vreme
Ponekod po Sloveniji megla v pasovih zmanjšuje vidljivost. Prilagodite hitrost!
Omejitve za tovorna vozila
Po Sloveniji velja med prazniki omejitev za tovorna vozila z največjo dovoljeno maso nad 7,5 ton:
- danes, 1. 1., od 8. do 22. ure;
- v nedeljo, 2. 1., od 8. do 22. ure.
Od 30. decembra je v veljavi sprememba omejitve za tovorna vozila nad 7,5 ton. Več.
Dela
Na primorski avtocesti je ponovno odprt priključek Črni Kal v obe smeri.

        POROČILO:
            Podatki o prometu.
Zaradi del je na vzhodni mariborski obvoznici v obe smeri zaprt prehitevalni pas med razcepom Dragučova in priključkom Pesnica.
Cesta čez prelaz Vršič je prevozna le za osebna vozila z zimsko opremo.
Ponekod po državi megla zmanjšuje vidljivost.

    
    PRIME

In [ ]:
res = pline(prompt, max_new_tokens=512)
print(res[0]["generated_text"])



Napiši poročilo iz podanih informacij. V primeru nujnih informacij jih obvezno vključi v poročilo, kot v primerih. 
Podanih imaš nekaj primerov.

    PRIMER 4: 

        INFORMACIJE:
            Nujne informacije: nan

            Informacije: Vreme
Ponekod po Sloveniji megla v pasovih zmanjšuje vidljivost. Prilagodite hitrost!
Omejitve za tovorna vozila
Po Sloveniji velja med prazniki omejitev za tovorna vozila z največjo dovoljeno maso nad 7,5 ton:
- danes, 1. 1., od 8. do 22. ure;
- v nedeljo, 2. 1., od 8. do 22. ure.
Od 30. decembra je v veljavi sprememba omejitve za tovorna vozila nad 7,5 ton. Več.
Dela
Na primorski avtocesti je ponovno odprt priključek Črni Kal v obe smeri.

        POROČILO:
            Podatki o prometu.
Zaradi del je na vzhodni mariborski obvoznici v obe smeri zaprt prehitevalni pas med razcepom Dragučova in priključkom Pesnica.
Cesta čez prelaz Vršič je prevozna le za osebna vozila z zimsko opremo.
Ponekod po državi megla zmanjšuje vidljivost.

    
    PRIME

In [31]:
print("Zaradi gorečega vozila je zaprta gorenjska avtocesta med Brezjem in predorom Ljubno proti Ljubljani. Obvoz po vzporedni regionalni cesti med priključkoma Brezje in Podtabor.")

print(test_content)

Zaradi gorečega vozila je zaprta gorenjska avtocesta med Brezjem in predorom Ljubno proti Ljubljani. Obvoz po vzporedni regionalni cesti med priključkoma Brezje in Podtabor.
Podatki o prometu.
Gorenjska avtocesta proti Ljubljani je zaradi gorečega vozila zaprta med priključkoma Brezje in Podtabor. Obvoz je po vzporedni regionalni cesti.
Danes do 21-ih velja prepoved prometa tovornih vozil, težjih od 7 ton in pol.

